In [ ]:
#packages

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.optimize import curve_fit
import pandas as pd


#centeroid finder
def centeroid(pic):
    x_len, y_len = pic.shape
    c_cord = np.array([0,0])
    weight = np.sum(pic)
    for i in range(x_len):
        for j in range(y_len):
            c_cord[0] += pic[i][j]*i
            c_cord[1] += pic[i][j]*j       
    return c_cord[0]//weight , c_cord[1]//weight

#crop_around
def crop_around(stars_img, star_x, star_y):
    return stars_img[star_y-50: star_y+50].T[star_x-50: star_x+50].T

#bad pixel removal
def bad_pix_rem(star_img):
    for i in range(1,99):
        for j in range(1,99):
            nhood=np.array([star_img[i-1][j],star_img[i-1][j-1],star_img[i-1][j+1],star_img[i][j-1],star_img[i][j+1],
                            star_img[i+1][j],star_img[i+1][j-1],star_img[i+1][j+1]])
            nhood_med=np.median(nhood)
            if star_img[i][j]>200 + nhood_med:
                star_img[i][j]=nhood_med
    return star_img

#gaussian function for curve_fit
def gauss(x,a,b):
    asec_per_pix=0.047
    return np.exp(-a * (x-centeroid(star_0_img)[0]*asec_per_pix)**2)+b

#initializing_data 

starsss=pd.read_csv("stars.csv")
loc_x = np.array(starsss["x0"].tolist())
loc_y = np.array(starsss["y0"].tolist())

img = fits.open('light-r_20C-2023_10_10-exp00.03.00.000-1x1_High_3.fit')
img_data = img[0].data
img_len_x, img_len_y = img_data.shape
asec_per_pix=0.047
sigma_hist=np.array([])
delta_sigma_hist=np.array([])

for i in range(len(loc_x)):
    star_0_img=bad_pix_rem(crop_around(img_data, int(loc_x[i]), img_len_y - int(loc_y[i])))

    #corrected_pic_print
    #plt.imshow(star_0_img, cmap='gray')
    #plt.colorbar()
    #plt.show()

    #curve_fit_setup

    x_data=np.arange(0,100)
    intensity=star_0_img[0:100][int(centeroid(star_0_img)[0])]
    max_intensity=intensity.max()
    intensity=intensity/max_intensity

    def gauss(x,a,b):
        return np.exp(-a * (x-centeroid(star_0_img)[0])**2)+b

    popt, pcov=curve_fit(gauss,x_data,intensity)
    delta_a=(pcov[0][0])**(1/2)
    sigma_pix=(popt[0]**(-1)/2)**(0.5)
    delta_sigma_pix=(sigma_pix**3)*delta_a
    sigma=sigma_pix*asec_per_pix
    delta_sigma=delta_sigma_pix*asec_per_pix
    sigma_hist=np.append(sigma_hist,sigma)
    delta_sigma_hist=np.append(delta_sigma_hist,delta_sigma)
    #plt.scatter(x_data,intensity)
    #plt.show()

plt.hist(sigma_hist)
plt.show()